# 爬取图片

In [4]:
import requests
from bs4 import BeautifulSoup as bs
import re
import os
import numpy as np

In [211]:
url = "https://yande.re/post?tags=wallpaper"
r = requests.get(url)
r.status_code

200

In [212]:
yande = r.text
soup = bs(yande, "html.parser") # 获得汤

In [185]:
# raw_link = soup.find_all('span','plid') #这个方法被淘汰了
#def get_link(raw_link):
    #link = raw_link.string[4:]
    #return link
#link = list(map(get_link, raw_link))

### 获取所有预览图的标签

`soup.find_all('img',attrs={"class":"preview"})`

In [215]:
# 获取了所有缩略图的链接
small_url = soup.find_all('img',attrs={"class":"preview"})
link = [l.get('src') for l in small_url]
for l in link[:5]:
    print(l)

https://assets.yande.re/data/preview/82/54/8254207b92bec804a6bfc4bc788edf77.jpg
https://assets.yande.re/data/preview/f5/61/f56133aebc008576b41b32ba1a44fdd3.jpg
https://assets.yande.re/data/preview/d6/be/d6becb17523fb77c327e2b2c3ead3f51.jpg
https://assets.yande.re/data/preview/78/69/78690f969a0315ad9ebed7453f6def89.jpg
https://assets.yande.re/data/preview/4e/fa/4efade46a747dec67015b4de7d650fc4.jpg


link 是一个储存了40个缩略图图片地址的列表

### 获得图片ID

In [229]:
small_url[0].parent

<a class="thumb" href="/post/show/551214"><img alt="Rating: Safe Score: 12 Tags: alice_schuberg armor sword sword_art_online sword_art_online_alicization tagme wallpaper User: Saturn_V" class="preview" height="84" src="https://assets.yande.re/data/preview/82/54/8254207b92bec804a6bfc4bc788edf77.jpg" style="margin-left: 0px;" title="Rating: Safe Score: 12 Tags: alice_schuberg armor sword sword_art_online sword_art_online_alicization tagme wallpaper User: Saturn_V" width="150"/><span class="plid">#pl https://yande.re/post/show/551214</span></a>

In [232]:
small_url[0].parent.get('href')

'/post/show/551214'

前面加上前缀就能进入展示页面

https://yande.re/post/show/551214

In [233]:
pic_id = [url.parent.get('href').split('/')[-1] for url in small_url]
pic_id[:5]

['551214', '551108', '551107', '551106', '550923']

### 获取图片码

In [222]:
link[0].strip('.jpg').split('/')[-1]

'8254207b92bec804a6bfc4bc788edf77'

In [224]:
pic_code = [l.strip('.jpg').split('/')[-1] for l in link]
pic_code[:5]

['8254207b92bec804a6bfc4bc788edf77',
 'f56133aebc008576b41b32ba1a44fdd3',
 'd6becb17523fb77c327e2b2c3ead3f51',
 '78690f969a0315ad9ebed7453f6def89',
 '4efade46a747dec67015b4de7d650fc4']

## 图片爬取

https://files.yande.re/sample/pic_code/pic_id.jpg

需要补上的有pic_code和pic_id



## 根据tag获取图片id和code

https://yande.re/post?tags=wallpaper
https://yande.re/post?page=1&tags=wallpaper

In [1]:
# 获得第1页到第5页图片的id和code
# tag = "wallpaper"
# pages 获取某几页的图片信息 pages = np.arange
def get_id_code(tag,page_range=(1,1)):
    
    iters = page_range[1] - page_range[0] + 1 # 迭代次数
    init_page = page_range[0] # 起始页面
    
    print("获得第{}页到第{}页{}图片的ID和CODE".format(page_range[0],page_range[1],tag))
    
    pic_code = [] # 用来储存图片CODE
    pic_id = [] # 用来储存图片的ID
    good = 0 # 获得页面信息成功计数
    bad = 0 # 获得页面信息失败计数
    
    for i in range(iters):
        
        page = init_page + i
        
        url = "https://yande.re/post?page={}&tags=".format(page)+tag  
        print(url)
        try:    
            print("正在获取第{}页的图片信息".format(page))
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            good += 1
        except:
            print("获取第{}页时图片信息时出错".format(page))
            bad += 1
            continue
          
        text = r.text
        soup = bs(text, "html.parser")
        
        small_url = soup.find_all('img',attrs={"class":"preview"}) # 获得所有缩略图标签
        
        pic_code += [l.get('src').strip('.jpg').split('/')[-1] for l in small_url] # 获得图片码
        pic_id += [l.parent.get('href').split('/')[-1] for l in small_url] # 获得图片id
    print("获取完毕，共获取了{}页图片信息，失败了{}页".format(good,bad))
    return pic_code, pic_id


### 根据下载链接下载图片

In [2]:
#输入：储存的文件路径，图片名称，图片地址
def download_picture(root,pic_name,url):
    #path = root + pic_name + '.' + url.split('.')[-1] # 多了个.jpg
    path = root + pic_name
    try:
        if not os.path.exists(root):
            os.makedirs(root)
        if not os.path.exists(path):
            r = requests.get(url, timeout=10)
            with open(path, 'wb') as f:
                f.write(r.content)
                f.close()
                return '图片保存成功'
        else:
            return '图片重名或图片已存在'
    except:
        return '图片爬取失败'


### 获取下载链接(已暂时淘汰)

In [236]:
def get_download_url(url):
    try:
        r = requests.get(url, timeout=30) # 获取url网页信息，如时间超过30秒，引发异常
        r.raise_for_status() # 若状态不是200，引发异常
        r.encoding = r.apparent_encoding # 用更加靠谱的编码方式
    except:
        return None
    
    text = r.text
    soup = bs(text, "html.parser")
    #download_url = soup.find_all('a',id ='highres')[0].get('href') #中质量版
    download_url = soup.find_all('img',id ='image')[0].get('src')
    return download_url


In [238]:
def 淘汰代码():
    # 不淘汰的时候把这个函数头删去
    root = 'D://pics//tag//'
    name = 'pic'
    for i,l in enumerate(link[:3]):
        pic_name = name + str(i+1)

        download_url = get_download_url(l)
        if download_url==None:
            print("第{}张图片的下载地址获取失败".format(i+1))
            continue
        print("已获取第{}张图片的下载地址".format(i+1))

        info = download_picture(root,pic_name,download_url)
        print("第{}张".format(i+1)+info)


In [14]:
import time
t_start=time.time()

tag="uncensored"
pic_code, pic_id = get_id_code(tag,page_range=(13,15))
t_end=time.time()
t_total = round(t_end- t_start,2)

print("共花费{}s".format(t_total))

获得第13页到第15页uncensored图片的ID和CODE
https://yande.re/post?page=13&tags=uncensored
正在获取第13页的图片信息
https://yande.re/post?page=14&tags=uncensored
正在获取第14页的图片信息
https://yande.re/post?page=15&tags=uncensored
正在获取第15页的图片信息
获取完毕，共获取了3页图片信息，失败了0页
共花费13.25s


In [15]:
len(pic_code)

120

In [16]:
main_url = "https://files.yande.re/sample/"
root = 'D://pics//' + tag + '//'

t_start = time.time()
fail = 0 # 记录保存失败的次数
success = 0
for i,(code,pid) in enumerate(zip(pic_code,pic_id)):
    t_mid = time.time()
    
    pic_name = pid + ".jpg" # 给图片命名
    download_url = main_url + code + '/' + pic_name # 图片下载链接
    info = download_picture(root,pic_name,download_url) # 下载图片
    
    t_end=time.time()
    t_total = t_end - t_start # 花费的总时间
    t_iter = t_end - t_mid # 这次迭代花费的时间
    
    msg1 = "第{}张".format(i+1)+info+"!"
    msg2 = "总时间:{:5.2f}s,这张图片花费时间:{:2.2f}s。".format(t_total,t_iter)
    if info == '图片保存成功':
        success += 1
    else:
        fail += 1
    msg3 = "成功{}张，失败{}张".format(success,fail)
    print("\r",msg1+msg2+msg3,end="",flush=True)
    
    
print("\n下载完毕，共保存了{}张图片，失败了{}张".format(success,fail))


 第120张图片保存成功!总时间:480.15s,这张图片花费时间:15.53s。成功34张，失败86张
下载完毕，共保存了34张图片，失败了86张


统计已保存图片数量

日期等


In [339]:
import time

print("倒计时程序")

for x in range(5,-1,-1):

    mystr = str(x) + "秒"
    print("\r",mystr,end="",flush=True)
    print("nihao")
    time.sleep(1)

    

倒计时程序
 5秒nihao
 4秒nihao
 3秒nihao
 2秒nihao
 1秒nihao
 0秒nihao


In [329]:
print("\r",object,end="",flush=True)

16

In [11]:
import os ,os.path ,time

def rename(file,keyword):
    ''' file: 文件路径    keyWord: 需要修改的文件中所包含的关键字 '''
    start =time.clock()
    os.chdir(file) # 
    items = os.listdir(file)
    print(os.getcwd())
    for name in items :
        print(name)
        # 遍历所有文件
        if not os.path.isdir(name):
            if keyword not in name :
                new_name = name+'.jpg'
                os.renames(name,new_name)
        else:
            rename(file + '\\' + name, keyword)
            os.chdir('...')      
    print('-----------------------分界线------------------------')
    items = os.listdir(file)
    for name in items:
        print(name)
        
#rename('D://pics//' + tag + '//', '.jpg')